<a href="https://colab.research.google.com/github/Thcataclismo/Pipeline-skcikit-learn/blob/main/Pipelines_no_Scikit_Learn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Pipelines no Scikit-Learn**

https://www.kaggle.com/datasets/jamesleslie/titanic-cleaned-data

### A classe Pipeline é uma funcionalidade do Scikit-Learn que ajuda criar códigos que possuam um padrão que possa ser facilmente entendido e compartilhando entre times de cientista e engenheiro de dados.

In [ ]:
pip install --upgrade category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 939 kB/s 


In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from category_encoders import OneHotEncoder
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate


In [ ]:
# lendo o dataset
df = pd.read_csv("/content/drive/MyDrive/archive (3)/train_clean.csv")

In [ ]:
# retirando colunas com nome, ingresso e cabine dos conjuntos
df.drop(["Name", "Ticket", "Cabin"], axis=1, inplace=True)

In [ ]:
# dividindo em conjunto de treino e test
X_train, X_test, y_train, y_test = train_test_split(df.drop(['Survived'], axis=1), 
                                                    df['Survived'], 
                                                    test_size=0.2, 
                                                    random_state=42)

In [ ]:
# criando o modelo usando pipeline
model = Pipeline(steps=[
    ('one-hot encoder', OneHotEncoder()),
    ('imputer', SimpleImputer(strategy='mean')),
    ('tree', DecisionTreeClassifier(max_depth=3, random_state=0))
])

In [ ]:
# treinando o modelo
model.fit(X_train, y_train)
train_score = model.score(X_train, y_train)

### quando chamamos o método fit, todos os passos definidos no Pipeline serão executados na ordem em que aparecem.



In [ ]:
# avaliando o modelo
test_score = model.score(X_test, y_test)

In [ ]:
print("Train score: {}".format(train_score))
print("Test score: {}".format(test_score))

Train score: 0.8356741573033708
Test score: 0.8156424581005587


### No Pipeline acima, primeiro aplicamos o OneHotEncoder do pacote category_encoders. Esse transformador irá pegar todas as variáveis categóricas dos dados e aplicar a transformação. O segundo passo do Pipeline é a imputação de valores faltantes em variáveis númericas pela substituição dos valores faltantes pela média. O terceiro e último passo é uma árvore de decisão que será treinada nos dados pré-processados nos passos anteriores.

# ***Cross-Validation com Pipelines***

### Uma grande vantagem de usar o Pipeline é a possibilidades de incluí-los no esquema de cross-validation do seu modelo. 

In [ ]:
# validando o modelo usando 5-fold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
results = cross_validate(model, X=df.drop(['Survived'], axis=1), y=df['Survived'], cv=kfold)
print("Average accuracy: %f (%f)" %(results['test_score'].mean(), results['test_score'].std()))

Average accuracy: 0.823803 (0.024182)


### Perceba como, uma vez que você encapsulou todo o seu modelo dentro de um Pipeline, fica muito mais fácil validá-lo de forma correta e confiável. De forma bastante explícita, com o Pipeline o seu modelo não consiste apenas do algoritmo, mas também de todo o pré-processamento usado antes de aplicar o algoritmo nos dados.

# ***Grid-Search com Pipelines*** (hiperparâmetros)

### Uma vez que o pipeline está criado, podemos também facilmente realizar uma tunagem de hiperparâmetro utilizando o GridSearchCV. O pipeline basicamente se torna o nosso modelo, logo, basta apenas definirmos o tipo de validação desejada e os parâmetros que deverão ser tunados. 

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# Tunando hiperparâmetros com 5-fold cross-validation e pipelines
parameters = {'tree__max_depth': [3, 4, 5]}
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
grid = GridSearchCV(model, param_grid=parameters, cv=kfold, n_jobs=-1)
grid.fit(X=df.drop(['Survived'], axis=1), y=df['Survived'])

GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('one-hot encoder', OneHotEncoder()),
                                       ('imputer', SimpleImputer()),
                                       ('tree',
                                        DecisionTreeClassifier(max_depth=3,
                                                               random_state=0))]),
             n_jobs=-1, param_grid={'tree__max_depth': [3, 4, 5]})

In [ ]:
# qual o melhor parâmetro
grid.best_params_ 

{'tree__max_depth': 3}

# ***Pré-processando diferentes variáveis com ColumnTransfomer***

### ColumnTransformer(): Este transformador serve para especificar em qual coluna a transformação deve ser aplicada.



### Claramente, você precisará de tratamentos diferentes para cada uma delas. Enquanto na primeira podemos preencher valores nulos com a mediana e normalizar os dados, para a segunda podemos preencher os valores nulos com o valor mais frequente.

In [ ]:
from sklearn.compose import ColumnTransformer

In [ ]:
# pipeline para pré-processamento das variáveis Age e Fare
num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))
])

In [ ]:
# pipeline para pré-processamento das variáveis Sex e Embarked
cat_transformer = Pipeline(steps=[
    ('one-hot encoder', OneHotEncoder())
])


In [ ]:
# Compondo os pré-processadores
preprocessor = ColumnTransformer(transformers=[
    ('num', num_transformer, ['Age', 'Fare']),
    ('cat', cat_transformer, ['Sex', 'Embarked'])
])

In [ ]:
# criando o modelo usando pipeline
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('tree', DecisionTreeClassifier(max_depth=3, random_state=0))
])

In [ ]:
# Tunando hiperparâmetros com 5-fold cross-validation e pipelines
parameters = {'tree__max_depth': [3, 4, 5]}
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
grid = GridSearchCV(model, param_grid=parameters, cv=kfold, n_jobs=-1, return_train_score=True)
grid.fit(X=df.drop(['Survived'], axis=1), y=df['Survived'])

GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='median'))]),
                                                                         ['Age',
                                                                          'Fare']),
                                                                        ('cat',
                                                                         Pipeline(steps=[('one-hot '
                                                                                          'encoder',
                                                                                          OneHotEncoder())]),
             

### O Pipeline permite que você passe pra ele até um pré-processador customizado, inteiramente construído para as suas necessidades.